In [1]:
# Dependencies

import pandas as pd
import numpy as np
import glob
import os
import timeit                                # To check performance
from datetime import datetime, date, time, timedelta

In [2]:
# Export clean CSV file to master CSV file

tic = timeit.default_timer()                            # Monitor performance

ecobici_rides_df = pd.read_csv("dataset_export_files/ecobici_ride_data_for_sql_import.csv", index_col=None, header=0)

toc = timeit.default_timer()                            # Monitor performance
print(f'Time (in seconds) to export unified CSV file: {round(toc - tic, 2)}')

Time (in seconds) to export unified CSV file: 0.59


In [3]:
for col in ecobici_rides_df.columns:
    print(f'{col}, type: {type(ecobici_rides_df.loc[0, col])}')
    


Genero_Usuario, type: <class 'str'>
Edad_Usuario, type: <class 'numpy.float64'>
Ciclo_Estacion_Retiro, type: <class 'numpy.float64'>
Ciclo_Estacion_Arribo, type: <class 'numpy.int64'>
Usage_Timestamp, type: <class 'str'>
Duration(Min), type: <class 'numpy.float64'>


In [5]:
ecobici_rides_df_test = ecobici_rides_df.copy()
ecobici_rides_df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707039 entries, 0 to 707038
Data columns (total 6 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Genero_Usuario         707039 non-null  object 
 1   Edad_Usuario           707039 non-null  float64
 2   Ciclo_Estacion_Retiro  707039 non-null  float64
 3   Ciclo_Estacion_Arribo  707039 non-null  int64  
 4   Usage_Timestamp        707039 non-null  object 
 5   Duration(Min)          707039 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 32.4+ MB


In [8]:
ecobici_rides_df_test['Usage_Timestamp'] = pd.to_datetime(ecobici_rides_df_test['Usage_Timestamp'])
ecobici_rides_df_test.head()

,Genero_Usuario,Edad_Usuario,Ciclo_Estacion_Retiro,Ciclo_Estacion_Arribo,Usage_Timestamp,Duration(Min)
0,M,28.0,85.0,85,2010-02-16 12:45:37.427,3.1
1,M,35.0,78.0,63,2010-02-19 13:52:30.700,12.1
2,M,34.0,85.0,23,2010-02-21 14:30:46.390,26.8
3,M,46.0,20.0,17,2010-02-22 16:17:10.470,6.4
4,F,28.0,74.0,46,2010-02-23 18:51:57.057,14.9


In [15]:
ecobici_rides_df_test[['Edad_Usuario', 'Ciclo_Estacion_Retiro', 'Ciclo_Estacion_Arribo']] = ecobici_rides_df_test[['Edad_Usuario', 'Ciclo_Estacion_Retiro', 'Ciclo_Estacion_Arribo']].astype(int)
ecobici_rides_df_test
# df.a = df.a.astype(float)

,Genero_Usuario,Edad_Usuario,Ciclo_Estacion_Retiro,Ciclo_Estacion_Arribo,Usage_Timestamp,Duration(Min)
0,M,28,85,85,2010-02-16 12:45:37.427,3.1
1,M,35,78,63,2010-02-19 13:52:30.700,12.1
2,M,34,85,23,2010-02-21 14:30:46.390,26.8
3,M,46,20,17,2010-02-22 16:17:10.470,6.4
4,F,28,74,46,2010-02-23 18:51:57.057,14.9
...,...,...,...,...,...,...
707034,M,27,316,54,2021-12-05 09:29:40,13.3
707035,M,49,158,83,2021-12-05 09:24:30,10.8
707036,M,33,417,313,2021-12-05 08:55:46,12.1
707037,F,23,467,260,2021-12-05 08:54:12,33.7


In [14]:
# Export clean CSV file to master CSV file

tic = timeit.default_timer()                            # Monitor performance

ecobici_rides_df_test = pd.read_csv("dataset_export_files/ecobici_ride_data_for_sql_import.csv", index_col=None, header=0)

toc = timeit.default_timer()                            # Monitor performance
print(f'Time (in seconds) to export unified CSV file: {round(toc - tic, 2)}')
ecobici_rides_df_test

Time (in seconds) to export unified CSV file: 0.51


,Genero_Usuario,Edad_Usuario,Ciclo_Estacion_Retiro,Ciclo_Estacion_Arribo,Usage_Timestamp,Duration(Min)
0,M,28.0,85.0,85,2010-02-16 12:45:37.427,3.1
1,M,35.0,78.0,63,2010-02-19 13:52:30.700,12.1
2,M,34.0,85.0,23,2010-02-21 14:30:46.390,26.8
3,M,46.0,20.0,17,2010-02-22 16:17:10.470,6.4
4,F,28.0,74.0,46,2010-02-23 18:51:57.057,14.9
...,...,...,...,...,...,...
707034,M,27.0,316.0,54,2021-12-05 09:29:40,13.3
707035,M,49.0,158.0,83,2021-12-05 09:24:30,10.8
707036,M,33.0,417.0,313,2021-12-05 08:55:46,12.1
707037,F,23.0,467.0,260,2021-12-05 08:54:12,33.7


In [ ]:
ecobici_rides_df_testb.groupby(pd.Grouper(freq='M'))